EFFETS DU COVID SUR LECONOMIE DE LA ZONE EURO

Les pays de la Zone Euro étudiés sont ceux présents avant le 01/01/2023 :
Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Netherlands, Portugal, Slovakia, Slovenia, Spain, Greece

LIBRAIRIES ET CONSTANTES

In [40]:
# Librairies

import pandas as pd
import gzip
import numpy as np
import seaborn as sn
from datetime import datetime
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px

In [41]:
# Listes des pays de la zone euro (avant 2023) 

euro_area_19 = ['Austria', 'Belgium', 'Cyprus', 'Estonia', 'Finland', 'France', 'Germany', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Portugal', 'Slovakia', 'Slovenia', 'Spain', 'Greece']
euro_area_19_ISO = ['AT', 'BE', 'CY', 'EE', 'FI', 'FR', 'DE', 'IE', 'IT', 'LV', 'LT', 'LU','MT', 'NL', 'PT', 'SK', 'SI', 'ES', 'GR' , 'EL']

EXTRACTION DES DONNEES

In [ ]:
# Chemins et colonnes a utiliser

chemin_fichier_covid = '../Data/Data 2/owid-covid-data.csv'
chemin_europe = '../Data/Donnees_Europe/'

cols_BD = ['geo', 'TIME_PERIOD', 'OBS_VALUE']
cols_covid_eur = ['iso_code', 'continent', 'location', 'date', 'total_cases_per_million', 'new_cases_per_million', 'reproduction_rate', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'stringency_index', 'hospital_beds_per_thousand', 'human_development_index']
cols_pop = ['TIME_PERIOD', 'OBS_VALUE']


In [115]:
# Population de la Zone Euro 19 (avant 2023)

# Columns : ['TIME_PERIOD', 'OBS_VALUE']
# Periodicite : 'A'
# Unite : 'Millions of persons'
# Zone Geo : 'EuroArea19'
# Dates : 1995 -> 2022

population_Europe = pd.read_csv(chemin_europe+'Europe_Population.csv', sep = ',', skiprows=5, usecols=[0,1])
population_Europe.columns = ['TIME_PERIOD', 'OBS_VALUE']

bd_population_eur = pd.DataFrame(columns = cols_pop)

# On selectionne les donnees apres 2017
bd_population_eur.TIME_PERIOD = population_Europe.TIME_PERIOD.loc[population_Europe.TIME_PERIOD >= 2017]
bd_population_eur.OBS_VALUE = population_Europe.OBS_VALUE.loc[population_Europe.TIME_PERIOD >= 2017]

del population_Europe


In [ ]:
# Donnees relatives a la COVID

covid_Europe = pd.read_csv(chemin_fichier_covid, usecols= cols_covid_eur)

bd_covid_eur = covid_Europe.loc[covid_Europe.location.isin(euro_area_19)]
bd_covid_eur['date'] = pd.to_datetime(bd_covid_eur['date'])
bd_covid_eur['Quarter'] = bd_covid_eur['date']
bd_covid_eur['Quarter'] = pd.PeriodIndex(bd_covid_eur['Quarter'], freq='Q').to_timestamp()

bd_covid_eur2 = pd.DataFrame(columns = ['Quarter', 'total_cases_per_million', 'new_cases_per_million', 
                                        'var_new_cases_per_million', 'reproduction_rate', 'total_vaccinations_per_hundred', 
                                        'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 
                                        'stringency_index', 'var_STI', 'hospital_beds_per_thousand'])

quarter = []
mean_total_cases_per_million = []
mean_new_cases_per_million = []
mean_total_vaccinations_per_hundred = []
mean_people_vaccinated_per_hundred = []
mean_people_fully_vaccinated_per_hundred = []
mean_hospital_beds_per_thousand = []
mean_stringency_index = []
mean_reproduction_rate = []

for q in bd_covid_eur.Quarter.unique() :

    covid_q = bd_covid_eur.loc[bd_covid_eur.Quarter == q]
    if covid_q['location'].unique().shape[0] == len(euro_area_19) :
        quarter.append(q)
        mean_total_cases_per_million.append(covid_q.total_cases_per_million.values.mean())
        mean_new_cases_per_million.append(covid_q.new_cases_per_million.values.mean())
        mean_total_vaccinations_per_hundred.append(covid_q.total_vaccinations_per_hundred.values.mean())
        mean_people_vaccinated_per_hundred.append(covid_q.people_vaccinated_per_hundred.values.mean())
        mean_people_fully_vaccinated_per_hundred.append(covid_q.people_fully_vaccinated_per_hundred.values.mean())
        mean_hospital_beds_per_thousand.append(covid_q.hospital_beds_per_thousand.values.mean())
        mean_stringency_index.append(covid_q.stringency_index.values.mean())
        mean_reproduction_rate.append(covid_q.reproduction_rate.values.mean())

    else : 
        quarter.append(q)
        mean_total_cases_per_million.append(np.nan)
        mean_new_cases_per_million.append(np.nan)
        mean_total_vaccinations_per_hundred.append(np.nan)
        mean_people_vaccinated_per_hundred.append(np.nan)
        mean_people_fully_vaccinated_per_hundred.append(np.nan)
        mean_hospital_beds_per_thousand.append(np.nan)
        mean_stringency_index.append(np.nan)
        mean_reproduction_rate.append(np.nan)

bd_covid_eur2['Quarter'] = quarter
bd_covid_eur2['total_cases_per_million'] = mean_total_cases_per_million
bd_covid_eur2['new_cases_per_million'] = mean_new_cases_per_million

var_new_cases = []
var_new_cases.append(np.nan)
for i in range (1,len(bd_covid_eur2['new_cases_per_million'])) :
    val0 = bd_covid_eur2['new_cases_per_million'].values[i-1]
    val1 = bd_covid_eur2['new_cases_per_million'].values[i]
    var_new_cases.append((val1 - val0)/val0)

bd_covid_eur2['var_new_cases_per_million'] = var_new_cases

bd_covid_eur2['total_vaccinations_per_hundred'] = mean_total_vaccinations_per_hundred
bd_covid_eur2['people_vaccinated_per_hundred'] = mean_people_vaccinated_per_hundred
bd_covid_eur2['people_fully_vaccinated_per_hundred'] = mean_people_fully_vaccinated_per_hundred
bd_covid_eur2['hospital_beds_per_thousand'] = mean_hospital_beds_per_thousand
bd_covid_eur2['stringency_index'] = mean_stringency_index

var_STI = []
var_STI.append(np.nan)
for i in range (1,len(bd_covid_eur2['stringency_index'])) :
    val0 = bd_covid_eur2['stringency_index'].values[i-1]
    val1 = bd_covid_eur2['stringency_index'].values[i]
    var_STI.append((val1 - val0)/val0)

bd_covid_eur2['var_STI'] = var_STI

bd_covid_eur2['reproduction_rate'] = mean_reproduction_rate

del covid_Europe

In [ ]:
bd_covid_eur2

In [ ]:
# Donnees economiques (PIB, Chomage, Revenus par foyers, Inflation)

# PIB 

gdp_Europe = pd.read_csv(chemin_europe+'Europe_GDP.csv.gz', compression='gzip', sep = ',')

bd_gdp_eur = gdp_Europe.loc[gdp_Europe.geo == 'EA19']
bd_gdp_eur['Quarter'] = pd.PeriodIndex(bd_gdp_eur.TIME_PERIOD, freq='Q').to_timestamp()

gdp_q = bd_gdp_eur.loc[pd.Timestamp('2019-01-01') <= bd_gdp_eur.Quarter]
gdp_q = gdp_q.loc[pd.Timestamp('2022-07-01') >= gdp_q.Quarter].reset_index()

# ----------------------------------------------------------------------------------------------------------------

# Chomage 

chomage_Europe = pd.read_csv(chemin_europe+'Europe_Chomage.csv.gz', compression='gzip', sep = ',')

bd_chomage_eur = chomage_Europe.loc[chomage_Europe.geo == 'EA19']
bd_chomage_eur = bd_chomage_eur.loc[chomage_Europe.unit == 'PC_POP']
bd_chomage_eur['Quarter'] = pd.PeriodIndex(bd_chomage_eur.TIME_PERIOD, freq='Q').to_timestamp()

chom_q = bd_chomage_eur.loc[pd.Timestamp('2019-01-01') <= bd_chomage_eur.Quarter]
chom_q = chom_q.loc[pd.Timestamp('2022-07-01') >= chom_q.Quarter].reset_index()

# ----------------------------------------------------------------------------------------------------------------

# Revenus par foyers 

revenus_foyers_Europe = pd.read_csv(chemin_europe+'Europe_Revenus_foyers.csv.gz', compression='gzip', sep = ',')

bd_revenus_foyers_eur = revenus_foyers_Europe.loc[revenus_foyers_Europe.geo == 'EA19'] 
bd_revenus_foyers_eur['Quarter'] = pd.PeriodIndex(bd_revenus_foyers_eur.TIME_PERIOD, freq='Q').to_timestamp()

revenus_q = bd_revenus_foyers_eur.loc[pd.Timestamp('2019-01-01') <= bd_revenus_foyers_eur.Quarter]
revenus_q = revenus_q.loc[pd.Timestamp('2022-07-01') >= revenus_q.Quarter].reset_index()

# ----------------------------------------------------------------------------------------------------------------

# Variations du taux de chomage

var_chom = []
var_chom.append(np.nan)

for i in range (1,len(chom_q.OBS_VALUE)) :
    val0 = chom_q.OBS_VALUE.values[i-1]
    val1 = chom_q.OBS_VALUE.values[i]
    var_chom.append((val1 - val0)/val0)

# ----------------------------------------------------------------------------------------------------------------

# Data frame

bd_eco = pd.DataFrame(columns = ['Quarter', 'GDP', 'Chomage', 'Variation_chomage', 'Revenus'])
bd_eco['GDP'] = gdp_q.OBS_VALUE
bd_eco['Chomage'] = chom_q.OBS_VALUE
bd_eco['Variation_chomage'] = var_chom
bd_eco['Revenus'] = revenus_q.OBS_VALUE
bd_eco['Quarter'] = gdp_q.Quarter.values

# ----------------------------------------------------------------------------------------------------------------

del chomage_Europe, gdp_Europe, revenus_foyers_Europe
del bd_chomage_eur, bd_gdp_eur, bd_revenus_foyers_eur
del gdp_q, chom_q, revenus_q

In [107]:
# Extractions des données

hdi_Europe = pd.read_csv(chemin_europe+'Europe_human-development-index.csv')
bd_hdi_eur = hdi_Europe.loc[hdi_Europe.Entity.isin(euro_area_19)]

expenditure_travels_Europe = pd.read_csv(chemin_europe+'Europe_expenditure_travels.csv.gz', compression='gzip', sep = ',')
bd_expenditure_travels_eur = expenditure_travels_Europe.loc[expenditure_travels_Europe.geo == 'EA19']

inflation_Europe = pd.read_csv(chemin_europe+'Europe_inflation.csv.gz', compression='gzip', sep = ',')
bd_inflation_eur = inflation_Europe.loc[inflation_Europe.geo == 'EA19']

# -----------------------------------------------------------



# -----------------------------------------------------------




# household_elec_prices_Europe = pd.read_csv(chemin_europe+'Europe_household_elec_prices.csv.gz', compression='gzip', sep = ',')
# air_passengers_transport_Europe = pd.read_csv(chemin_europe+'Europe_air_passengers_transport.csv.gz', compression='gzip', sep = ',')
# freight_transport_Europe = pd.read_csv(chemin_europe+'Europe_freight_transport.csv.gz', compression='gzip', sep = ',')
# non_household_elec_prices_Europe = pd.read_csv(chemin_europe+'Europe_non_household_elec_prices.csv.gz', compression='gzip', sep = ',')

C:\Users\selim\AppData\Local\Temp\ipykernel_7212\1235805339.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bd_gdp_eur['Quarter'] = pd.PeriodIndex(bd_gdp_eur.TIME_PERIOD, freq='Q').to_timestamp()
C:\Users\selim\AppData\Local\Temp\ipykernel_7212\1235805339.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bd_revenus_foyers_eur['Quarter'] = pd.PeriodIndex(bd_revenus_foyers_eur.TIME_PERIOD, freq='Q').to_timestamp()
C:\Users\selim\AppData\Local\Temp\ipykernel_7212\1235805339.py:79: SettingWithCopyWarnin

array([  nan,  0.  , 11.11, ..., 27.59, 27.58, 23.61])

In [109]:
data = bd_quarters.merge(bd_covid_eur2[['Quarter','new_cases_per_million','var_new_cases_per_million','stringency_index', 'var_STI']], on='Quarter', how='inner')
data

,Quarter,GDP,Chomage,Revenus,Variation_chomage,new_cases_per_million,var_new_cases_per_million,stringency_index,var_STI
0,2020-01-01,-3.4,4.8,-2.84,0.000000,NaN,NaN,NaN,NaN
1,2020-04-01,-11.5,4.7,-10.17,-0.020833,18.569765,NaN,68.738178,NaN
2,2020-07-01,12.4,5.5,12.12,0.170213,33.399349,0.798588,47.765749,-0.305106
3,2020-10-01,-0.3,5.2,-2.82,-0.054545,316.409200,8.473514,62.321264,0.304727
4,2021-01-01,0.0,5.3,-1.20,0.019231,307.647588,-0.027691,69.787357,0.119800
5,2021-04-01,2.0,5.1,3.61,-0.037736,159.305795,-0.482181,60.458629,-0.133674
6,2021-07-01,2.3,4.9,3.91,-0.039216,179.157457,0.124614,44.760681,-0.259648
7,2021-10-01,0.6,4.6,-0.67,-0.061224,553.283409,2.088252,43.685681,-0.024017
8,2022-01-01,0.6,4.4,0.09,-0.043478,2148.362380,2.882933,38.823579,-0.111297
9,2022-04-01,0.9,4.4,0.52,0.000000,566.395471,-0.736359,20.842157,-0.463157


Graphiques

Données en fonction du temps 

In [123]:
plt.figure(figsize=(15,4))
sn.lineplot(data=data, x='Quarter', y = ['GDP', 'Revenus', 'var_new_cases_per_million','stringency_index'])
data.Quarter

ValueError: Length of list vectors must match length of `data` when both are used, but `data` has length 11 and the vector passed to `y` has length 4.

<Figure size 1500x400 with 0 Axes>